<a href="https://colab.research.google.com/github/OANInternational/aguapp/blob/master/estructurar_bbdd_y_subir_a_firebase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Proyecto para estructurar base de datos y subir a FIRBASE

# DIRECTORIO

utilizar google como directorio de trabajo


In [0]:
%cd

/root


In [0]:
%cd
%cd ..
%cd content

/root
/
/content


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
%cd drive/'Shared drives'/'Comité de Infraestructuras, Energía y Agua'/'Proyectos'/'Red de Reparadores de Bombas Manuales'/'Prototipo Automatización'/'BASES DE DATOS'

/content/drive/Shared drives/Comité de Infraestructuras, Energía y Agua/Proyectos/Red de Reparadores de Bombas Manuales/Prototipo Automatización/BASES DE DATOS


In [0]:
import yaml
spreadsheet_ids = yaml.load( open("sheets_parameters.yaml", "r" ))
#bombas de 2018, ultima version
ID1=str(spreadsheet_ids['ID1'])
#bombas de 2016, ultima version
ID2=str(spreadsheet_ids['ID2'])
#bombas de 2009, ultima version
ID3=str(spreadsheet_ids['ID3'])
#bombas final, datos de 2016 y 2009
ID4=str(spreadsheet_ids['ID4'])

# CONNECT TO FIREBASE

In [0]:
#!pip install firebase-admin

In [0]:
filename='firebase_parameters.json'

In [0]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

# Use a service account
cred = credentials.Certificate(filename)
firebase_admin.initialize_app(cred)

db = firestore.client()

In [0]:
col_pumps = db.collection('Pumps')

In [0]:
pumps = col_pumps.get()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 'Collection.get' is deprecated:  please use 'Collection.stream' instead.
  """Entry point for launching an IPython kernel.


In [0]:
for pump in pumps:
    print(u'{} => {}'.format(pump.id, pump.to_dict()))

# ABRIR LAS BBDD

## OPEN GOOGLE SHEETS

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
sheet_service = build('sheets', 'v4')

## IMPORTS

In [0]:
import plotly
import plotly.plotly as py
import numpy as np
import re

import pandas as pd


## OPEN FROM GOOGLE SHEET

In [0]:
ids = [ID1,ID2,ID3]
dfs = []
for ID in ids:
  #SPREADSHEET_ID = ID
  RANGE_NAME = 'Sheet1'

  gsheet = sheet_service.spreadsheets().values().get(spreadsheetId=ID, range=RANGE_NAME).execute()

  dfs.append(pd.DataFrame(gsheet.get('values', [])[1:],columns= gsheet.get('values', [])[0:1][0]))

df_2019=dfs[0]
df_2016=dfs[1]
df_2009=dfs[2]

In [0]:
dfs[1]["Etat de l'ouvrage"].unique()

array(['Fonctionnel', 'Panne', 'En panne', 'Tête de pompe', '2 pompes',
       'Privé', '1  Pompe', '', 'Pompe', 'Pompe non', 'Faible débit',
       '1 pompe', 'Kalale'], dtype=object)

In [0]:
[df.columns for df in dfs]
#df_2016.head(2)
#df_2009.head(2)

[Index(['Arrondissement', 'Village', 'Localite', 'Nom_Local', 'Longitude',
        'Latitude', 'xCoord', 'yCoord', 'rqtListe_Pompe.Modele_Pompe'],
       dtype='object'),
 Index(['N°', 'Arrondissement', 'Village', 'Localité', 'Hameau',
        'Etat de l'ouvrage', 'Redevance', 'Année de réalisation', 'Marque'],
       dtype='object'),
 Index(['Arrondissement', 'Village', 'Localité', 'Population', 'Type d'ouvrage',
        'Etat', 'Longitude', 'Latitude', 'Domaine d'installation'],
       dtype='object')]

So common data is:

* 2019 vs 2016: Arrondissement, Village, Localite, Marque

* 2019 vs 2009: Arrondissement, Village, Localite, Longitude, Latitude

* 2009 vs 2016: Arrondissement, Village, Localite

ONLY DATA:

* 2019: Nom_Local which is little identifier for more info on pumps

* 2016: Année de réalisation, , Redevace, Hameau, which would be kind of the same as Nom_Local

* 2009: Population, Type d'ouvrage

## Process the Data

### FUZZY TO COMPARE Localites

In [0]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 22.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [0]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [0]:
#Example of use
choices = df_2016['Localité'].tolist()
localite = df_2019.loc[20,'Localite']
process.extract(localite, choices, limit=5)


[('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou', 100),
 ('Ourarou Gberou kpanin', 90),
 ('Gnelkourarou', 90)]

In [0]:
#another one
word1=df_2019.loc[108,'Localite']
word2=df_2016.loc[150,'Localité']
print(word1,'\n',word2)

Kparissérou Gando 
 Kparisserou gando


In [0]:
fuzz.ratio(word1, word2)

88

In [0]:
word1=word1+'!'
fuzz.partial_ratio(word1, word2)

88

In [0]:
fuzz.ratio(word1, word2)

86

In [0]:
process.extract('Gando Gourou', df_2019['Localite'], limit=5)


[('Gando Gourou', 100, 329),
 ('Gourou', 90, 109),
 ('Gourou', 90, 110),
 ('Gourou', 90, 111),
 ('Dèh Gando', 86, 172)]

In [0]:
#last one
word1=df_2019.loc[329,'Localite']
word2=df_2016.loc[270,'Localité']
print(word1,'\n',word2)

Gando Gourou 
 Ganrou gando


In [0]:
fuzz.ratio(word1, word2)

42

In [0]:
fuzz.token_sort_ratio(word1, word2)

83

### LETS CLEAN THE DATA

#### Functions

In [0]:
#Marques des Pompes
def marques(x):
  marque='NA'
  
  if x['Marque']:
    if bool(re.search('ergnet',x['Marque'].lower())):
      marque='Vergnet'
    elif bool(re.search('ndia',x['Marque'].lower())):
      marque='India'
    elif bool(re.search('afri',x['Marque'].lower())):
      marque='Afridev'
  return marque

In [0]:
#Years
def getyear(x):
  year=None
  if x['Année de réalisation']:
    for s in x['Année de réalisation'].split():
      if s.isdigit():
         year=int(s)
    if year==None:
      return year
    elif year >2019:
      #print(year)
      year=2010
      return year
    elif year > 100:
      return int(year)
    elif year >70:
      year=int('19'+str(year))
      return int(year)
    else:
      year=None
    
  return year

In [0]:
'''
Function to change latitude/longitude data from degrees,minutes and seconds into decimals
'''
def decimals_lat_lon(x,lat):
  coord=None
  if x[lat]:
    if len(x[lat])>2:
      if len(re.split('[°\'"]', x[lat])) == 4 :
        
        #gets all the parts of the coordinates into different elements of a vector
        deg, minutes, seconds, direction =  re.split('[°\'"]', x[lat])
        
        #had to put replace commas because of data of 2009
        coord=(float(deg) + float(minutes.replace(",","."))/60 + float(seconds.replace(",","."))/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
      elif len(re.split('[°\'"]', x[lat])) > 0:
        coord=x[lat]
        print(coord)
  else:
    coord=None
    print(coord)
  return coord

#### Begin with 2019

In [0]:
#Create decimals from degrees latitude
df_2019['Latitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Latitude'),axis=1)

None
None


In [0]:
df_2019['Longitude_decimals']=df_2019.apply(lambda x: decimals_lat_lon(x,'Longitude'),axis=1)

None
None


In [0]:
df_2019_filtered = df_2019[['Arrondissement', 'Village','Localite', 'Longitude_decimals',
        'Latitude_decimals', 'rqtListe_Pompe.Modele_Pompe']]
df_2019_filtered.columns = ['Arrondissement', 'Village','Localite', 'Longitude',
        'Latitude', 'Marque']

In [0]:
df_2019_filtered['Marque']=df_2019_filtered.apply(lambda x: marques(x),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [0]:
l_2019_local = df_2019_filtered['Localite'].sort_values().tolist()

index = []
j=1
for i,local in enumerate(l_2019_local):
  if i == 0:
    j=1
    word = local+' '+str(j)
    word=word.replace(' ','_')
    index.append(word)
    continue
  if l_2019_local[i-1] == local:
      j+=1
      word = local+' '+str(j)
      word=word.replace(' ','_')
      index.append(word)
  else:
      j=1
      word = local+' '+str(j)
      word=word.replace(' ','_')
      index.append(word)

In [0]:
df_2019_filtered=df_2019_filtered.sort_values('Localite')

In [0]:
df_2019_filtered['index'] = index

In [0]:
df_2019_filtered=df_2019_filtered.set_index('index')

In [0]:
#convert to dictionary
dict_2019 = df_2019_filtered.to_dict('index')
dict_2019 = [{k: v} for k, v in dict_2019.items()]

## upload to Firebase

In [0]:
for data in dict_2019:
  id=list(data.keys())[0]
  doc_ref = db.collection(u'Pompes_2019').document(u''+str(id))
  doc_ref.set(data[id])